In [48]:
# Refubrished code 2025

# libraries
import cloudscraper # TBH, should have just used selenium if ik it would be this much of a pain.
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import random as rd
import time
from tqdm.notebook import tqdm
from typing import *
import re
import json
import os

In [ ]:
available_filters = {
    "manufacturers": ['Apple', 'Samsung', 'Google', 'OnePlus', 'Motorola', 'Sony', 'Xiaomi', 'Nokia', 'Honor', 'DOOGEE', 'Blackview', 'OSCAL', 'Nothing',
                       'Asus', 'HTC', 'Huawei', 'HMD', 'BlackBerry', 'ZTE', 'RedMagic', 'LG', 'Kyocera', 'Fairphone', 'Alcatel', 'BLU', 'Razer', 'realme', 'Essential',
                        'TCL', 'Orbic', 'CAT', 'RED', 'BOOX', 'Lenovo', 'OPPO', 'Microsoft', 'Acer', 'Garmin', 'Amazon', 'NOA', 'Meizu', 'nubia', 'GIGABYTE', 'Gionee', 'vivo', 'Panasonic',
                        'HP', 'Sony Ericsson', 'Maxwest', 'Verizon', 'Yota', 'Doro', 'T-Mobile', 'Sprint', 'Palm', 'Sanyo', 'Casio', 'VERZO', 'TAG Heuer', 'Xolo', 'VIZIO', 'Fujitsu', 'UMX',
                        'Garmin-Asus', 'Airo Wireless', 'TerreStar', 'Lumigon', 'FiGO', 'NIU', 'altek', 'Micromax', 'ARCHOS', 'Best Buy', 'Verykool', 'Notion Inc', 'Vertu', 'Sonim', 'Karbonn',
                        'ICEMOBILE', 'Emporia', 'Philips', 'Dell', 'ViewSonic', 'Toshiba', 'Barnes & Noble', 'Nvidia', 'PCD', 'Jolla', 'Eten', 'mobiado', 'i-mate', 'General Mobile', 
                        'Fusion Garage', 'INQ', 'Videocon', 'Coolpad', 'LAVA', 'Saygus', 'Yezz', 'Plum', 'Celkon', 'i-mobile', 'Spice Mobile', 'Zen Mobile', 'Velocity', 'COWON', 'Kogan',
                        'AT&T', 'VKMobile', 'Benq-Siemens', 'Helio', 'Haier', 'Lemon Mobiles', 'Handspring', 'Bird', 'Danger', 'WND', 'O2', 'Latte', 'Siemens', 'Pantech', 'Firefly Mobile',
                        'Cricket', 'Orange', 'Fly', 'Mitsubishi', 'MiTAC', 'Amoi', 'Sierra Wireless', 'Neonode', 'Sendo', 'Maxon', 'Hitachi', 'Sharp', 'NEC', 'Sagem', 'BenQ', 'Ericsson'],
                        
    "deviceType": { "Basic phone"    : "f[53][bp]=1221",
                    "Feature phone"  : "f[53][fp]=1222",
                    "Smartphone"     : "f[53][sp]=1223",
                    "Tablet"         : "f[53][t]=1612",
                    "Smartwatch"     : "f[53][sw]=2580"}
                    }

class scraper():
    '''
    Tbh idk why i made this overcomplicated if in the end i would just get all the data from phonearena, but yeah. It is what it is.

    Parameters
    ----------
    - manufacturers : Optional[List[str]]
        List of manufacturers to filter by. Default is None.
    - releaseYear : Optional[List[int]]
        List of release years to filter by. Default is None.
    - deviceType : Optional[List[str]]
        List of device types to filter by. Default is None.
    - max_timer : int
        Maximum time to wait between requests. Default is 4 seconds.
    '''
    def __init__(self, manufacturers: Optional[List[str]] = None, releaseYear: Optional[List[int]] = None, 
                 deviceType: Optional[List[str]] = None, max_timer: int = 4):
        '''
        NOTE: Only 20 filters can be applied at once on PhoneArena.
        '''
        self.URL = "https://www.phonearena.com/phones"
        self.max_timer = max_timer

        # Set filters of search
        if manufacturers:
            self.URL = self.URL + "/manufacturers/"
            manufacturers = ",".join([i.lower() for i in manufacturers])
            self.URL = self.URL + manufacturers
            self.usemanufacturers = True
            
        use_releaseYear = False
        if releaseYear:
            self.URL = self.URL + "?f[y]="
            if len(releaseYear) > 1:
                releaseYear = ','.join([str(i) for i in range(releaseYear[0], releaseYear[-1]+1)])
            else:
                releaseYear = str(releaseYear[0])
            self.URL = self.URL + releaseYear
            use_releaseYear = True

        if deviceType:
            if use_releaseYear:
                self.URL = self.URL + "&"
            else:
                self.URL = self.URL + "?"
            deviceType = "&".join([available_filters["deviceType"][i] for i in deviceType])
            self.URL = self.URL + deviceType

        # Where to store data for each phone
        self.gathered_data = {}
        self.driver = cloudscraper.create_scraper() # Initialize!

    def scape_phone_data(self):
        '''
        Scrapes data from a single phone page
        '''
        # Get release date
        widget_spec = self.phonespectable.find("div", attrs={"class" : "widgetQuickSpecs"})
        try:
            release_date = widget_spec.find("div", attrs={"class":"released specs-element"}).find("span", attrs={"class":"specs-element-desc"}).text.strip()
        except:
            release_date = "No information"

        self.temp["Release date"] = release_date

        # specifications data
        full_spec = self.phonespectable.find("section", attrs={"class" : "page__section page__section_specs"})
        spec_tables = full_spec.find_all("div", attrs={"class" : "widgetSpecs"})

        for tabel in spec_tables:
            # gets the category of the table
            category = tabel.find("th").text.strip()
            category_data = {}
            for item in tabel.find_all("tr", attrs={"class":"specs-table-title"}):
                attr = item.find("th").text.strip()
                val = item.find("td").text.strip()    
                category_data[attr] = val
            self.temp[category] = category_data

    def scrape_result_page(self):
        '''
        Scrapes a single result page for phone links and gathers data for each phone.
        '''
        
        result = self.soup_result_page.find("div", attrs={"class": "results"})

        phone_result = result.find_all("div", attrs = {"class" : "tile-phone"})

        # Iterate through each phone on the page
        for phone in tqdm(phone_result, desc="Scraping phones"):
            self.temp = {}  # Temporary storage each phones data
            phoneName = phone.find("a", attrs = {"class" : "tile-title"}).text
            phoneHREF = phone.find("a", attrs = {"class" : "tile-title"}).get("href")
            r = self.driver.get(phoneHREF)  # Get phone page
            time.sleep(rd.uniform(1,self.max_timer))
            self.phonespectable = BeautifulSoup(r.content, "html5lib")  # Get soup of the phone page
            self.scape_phone_data()

            # Store temp to main data dictionary
            self.gathered_data[phoneName] = self.temp


    def start(self, save_path: str = "temp.py", saveas_csv: bool = False):
        '''
        Starts the scraping process.

        Parameters
        ----------
        - save_path : str
            Path to save the scraped data. Default is "temp.py".
        - saveas_csv : bool
            Whether to save the data as a CSV file. Default is False.
        '''

        # Get first page
        r = self.driver.get(self.URL)
        time.sleep(rd.uniform(1,self.max_timer))
        self.soup_result_page = BeautifulSoup(r.content, "html5lib")

        # will get any existing max page number, if none, will just scrape the one page
        try:
            max_page = int(self.soup_result_page.find("nav", attrs = {"data-target":"finder-content"}).find_all("li", attrs = {"class": "item"})[-2].text)
            
        except:
            max_page = 1

        pbar = tqdm(total=max_page, desc="Scraping pages")

        # Scrape first page
        self.scrape_result_page()
        pbar.update(1)

        # if multiple pages, scrape the rest
        if max_page>1:
            # Adjust reference URL
            if self.usemanufacturers:
                # Ts if manufacturers were used as filter
                temp = re.search(r'manufacturers/[a-zA-Z,]+', self.URL, re.IGNORECASE)[0]
            else:
                temp = re.search(r'/phones', self.URL, re.IGNORECASE)[0]

            for page in range(2, max_page+1):
                newURL = self.URL.replace(temp, temp + f"/page/{page}")
                r = self.driver.get(newURL)
                time.sleep(rd.uniform(1,self.max_timer))
                self.soup_result_page = BeautifulSoup(r.content, "html5lib")
                self.scrape_result_page()
                pbar.update(1)

        pbar.close()
        # Save data
        if os.path.dirname(save_path):
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
        with open(f'{save_path}.json', 'w') as f:
            json.dump(self.gathered_data, f, indent=4)

        if saveas_csv:
            df = pd.DataFrame.from_dict(self.gathered_data, orient='index')
            unpacked_df = df.pop("Release date")
            unpacked_columns = [i for i in df.columns.values]
            for collumn in unpacked_columns:
                temp = pd.json_normalize(df[collumn])
                temp.columns = [f"{collumn} - {subcol}" for subcol in temp.columns]
                temp.index = df.index
                unpacked_df = pd.concat([unpacked_df, temp], axis=1)
            unpacked_df.to_csv(f'{save_path}.csv')

In [75]:
run = scraper(manufacturers=["Sony"], max_timer=3)

In [ ]:
run.start(save_path="EXAMPLE/Sony", saveas_csv=True)

Scraping pages:   0%|          | 0/2 [00:00<?, ?it/s]

Scraping phones:   0%|          | 0/60 [00:00<?, ?it/s]

https://www.phonearena.com/phones/manufacturers/sony/page/2


Scraping phones:   0%|          | 0/52 [00:00<?, ?it/s]